Code portions:
- All Pale chapters (through 23.1)
- Reasonably "optimal" preprocessing parameters
- fine-tuned embedding model
- ElasticSearch container Document Store
- Iterate and add other series embeddings?

In [1]:
%%bash

nvidia-smi


Mon Mar  6 02:38:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-ozgzkx02/farm-haystack_8b2b1a44a855441da730d32c82e30ea7
  Resolved https://github.com/deepset-ai/haystack.git to commit 1a421669786c80feee5ef17511202529c9c30c84
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-ozgzkx02/farm-haystack_8b2b1a44a855441da730d32c82e30ea7


In [3]:
import os
import pickle
import logging
import time
from haystack.document_stores import ElasticsearchDocumentStore
from haystack import Document
from haystack.nodes import PreProcessor, EmbeddingRetriever, Seq2SeqGenerator, TransformersSummarizer, FARMReader
from haystack.pipelines import GenerativeQAPipeline, ExtractiveQAPipeline, SearchSummarizationPipeline
from haystack.utils import launch_es


In [4]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [5]:
print(os.getcwd())
os.chdir('./drive/MyDrive/pale-companion-files/es-experimentation/')

print(os.getcwd())

/content
/content/drive/MyDrive/pale-companion-files/es-experimentation


In [6]:
MODEL_LOCATION = "../finetune/twig_otherverse_parahumans_adapted"
PALE_CHAPTERS_FNAME = "../chapter_fmt_list.pkl"

In [7]:
with open(PALE_CHAPTERS_FNAME,'rb') as f:
    all_chapters = pickle.load(f)
# chapters = [i for i in all_chapters if int(i['meta']['arc_number']) < FINAL_TEST_CHAPTER]
print(f"Testing with {len(all_chapters)} chapters")

Testing with 307 chapters


In [8]:
preprocessor = PreProcessor(
    split_by='word',
    split_length=250,
    split_overlap=20,

    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_respect_sentence_boundary=True,
    progress_bar=True, 
    add_page_number=True
)
docs = preprocessor.process(all_chapters)
print(f"We will be working with {len(docs)} documents from {len(all_chapters)} chapters")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Preprocessing:   0%|          | 0/307 [00:00<?, ?docs/s]

We will be working with 15203 documents from 307 chapters


In [9]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
# from haystack.utils import launch_es
# This is what we would use if we're outside of Colab - instead we have to do something else
# launch_es()


In [10]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2


In [19]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch


In [20]:
import time

time.sleep(30)


In [17]:
# from haystack.utils import launch_es
# launch_es()


In [22]:
# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
print(host)

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="pale_index",
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/elasticsearch/connection/http_urllib3.py", line 255, in perform_request
    response = self.pool.urlopen(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    retries = retries.increment(
  File "/usr/local/lib/python3.8/dist-packages/urllib3/util/retry.py", line 525, in increment
    raise six.reraise(type(error), error, _stac

localhost


ConnectionError: ignored

In [ ]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=MODEL_LOCATION,
    model_format="sentence_transformers",
    max_seq_len=500,
    progress_bar=True,
)

document_store.update_embeddings(retriever)

In [ ]:
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa", num_beams=8, max_length=500)
# Eventually with want to tweak the above
pipe = GenerativeQAPipeline(generator, retriever) # We specify the params later

In [ ]:
TEST_QUESTION = "Who is Avery Kelly?"

pipe.run(
                query = TEST_QUESTION,
                params = {
                    "Retriever": {"top_k":10},
                    # "Generator": {"max_length": maxlength, "num_beams": nbeam}
                }
            )